Copyright (c) Microsoft Corporation. Licensed under the MIT License. 

#Protect machine learning Models with Azure Percept SDK

**NOTE**
* Make sure to run each cell individually as some cells will require input parameters.
* Make sure you have provisioned Azure Percept MM service using the steps at https://go.microsoft.com/fwlink/?linkid=2156577.
* Ensure you have locally cloned the azure-percept-advanced-development repository.

This notebook shows how to 
* protect a model in-transit and at-rest with Azure Percept SDK
* how to deploy the protected model to AzureEyeModule on Azure Percept DK
* how to collect and protect retraining data.


In [ ]:
# Config AzureEyeModule running on your IoT edge device.

# Steps:
# 1. Navigate to the Azure Portal
# 2. Select IoT Hub
# 3. Select your IoT Hub instance
# 4. Select IoT Edge
# 5. Select your device
# 6. Select 'Set Modules'
# 7. Select azureeyemodule
# 8. Add the following 4 environment variables to the AzureEyeModule
    # AZURE_CLIENT_ID		    <Service Principal Client ID generated during MM server provisioning>
    # AZURE_CLIENT_SECRET   	<Service Principal Client Secret generated during MM server provisioning>
    # AZURE_TENANT_ID		    <Service Principal Tenant ID generated during MM server provisioning>
    # CONFIDENCE_THRESHOLD  	95 (The collected data whose confidence is below the theshold will be uploaded for retraining)

from IPython import display
display.Image("../imgs/azureeyemoduledetails.png")

In [ ]:
# Download a model package from Azure Percept public share
# This is done to show  an example of how to use the Azure Percept Model Management SDK
# In normal practice the model may be created and managed through a pipeline such as Azure Machine Learning (AML)
#
# Model Name: Person Detection
# Model Version: 13
!curl -L "https://aedsamples.blob.core.windows.net/vision/aeddevkitnew/person-detection-retail-0013.zip" --output model.zip

# Declare the model information
MODEL_NAME = 'person-detection'
MODEL_VERSION = '0013'


In [ ]:
# Declare the variables that defines the Azure Service Principal and model management server URL to be used for model protection. 
# These values were established while provisioning the Azure Percept MM service, see https://go.microsoft.com/fwlink/?linkid=2156577

AZURE_CLIENT_ID='<ENTER YOUR SERVICE PRINCIPAL CLIENT ID>'
AZURE_CLIENT_SECRET='<ENTER YOUR SERVICE PRINCIPAL CLIENT SECRET>'
AZURE_TENANT_ID='<ENTER YOUR SERVICE PRINCIPAL TENANT ID>'
SERVER_URL='<ENTER YOUR AZUURE PERCEPT MM SERVER URL>'

# Install Azure Percept SDK.
!pip install sczpy

# Protect the model with Azure Percept SDK 
import sczpy
import os

os.environ["AZURE_CLIENT_ID"] = AZURE_CLIENT_ID
os.environ["AZURE_CLIENT_SECRET"] = AZURE_CLIENT_SECRET
os.environ["AZURE_TENANT_ID"] = AZURE_TENANT_ID

client = sczpy.SCZClient(SERVER_URL)
client.register_model(MODEL_NAME, MODEL_VERSION)
client.encrypt(MODEL_NAME, MODEL_VERSION, "model.zip", "model.enc.zip")
client.upload_model(MODEL_NAME, MODEL_VERSION, "model.enc.zip")

# clean up the local model packages
os.remove("model.zip")
os.remove("model.enc.zip")

In [ ]:
# Perform module twin update on azureeyemodule to run the following functions:
# 1. Download and run inference with the protected model
# 2. Upload the encrypted retraining data to the storage account every 60 seconds

!pip3 install azure-iot-hub
import sys
from azure.iot.hub import IoTHubRegistryManager
from azure.iot.hub.models import Twin, TwinProperties

# Incorporate Iothub connection string and the default module name
# Go to Https://portal.azure.com
# Select your IoTHub
# Click on Shared access policies
# Click service on right
# Copy the iothub connection string primary key to CONNECTION_STRING
# Copy the device name given to the Azure Percept DK device to DEVICE_ID

CONNECTION_STRING = '<ENTER YOUR CONNECTION STRING>'
DEVICE_ID = '<ENTER YOUR DEVICE NAME>'
MODULE_ID = "azureeyemodule"

try:
    iothub_registry_manager = IoTHubRegistryManager(CONNECTION_STRING)

    module_twin = iothub_registry_manager.get_module_twin(DEVICE_ID, MODULE_ID)
    print ( "" )
    print ( "Module twin properties before update    :" )
    print ( "{0}".format(module_twin.properties) )

    # Update module twin 
    twin_patch = Twin()
    twin_patch.properties = TwinProperties(desired={"SecureAILifecycleEnabled": True, 
                                                    "SCZ_MODEL_NAME": MODEL_NAME, 
                                                    "SCZ_MODEL_VERSION": MODEL_VERSION, 
                                                    "SCZ_MM_SERVER_URL": SERVER_URL,
                                                    "DownloadSecuredModelFromMMServer": True,
                                                    "RetrainingDataCollectionEnabled": True,
                                                    "RetrainingDataCollectionInterval": 60})
    updated_module_twin = iothub_registry_manager.update_module_twin(
        DEVICE_ID, MODULE_ID, twin_patch, module_twin.etag
    )
    print ( "" )
    print ( "Module twin properties after update     :" )
    print ( "{0}".format(updated_module_twin.properties) )

except Exception as ex:
    print ( "Unexpected error {0}".format(ex) )
except KeyboardInterrupt:
    print ( "IoTHubRegistryManager sample stopped" )

In [ ]:
# The settings will get pushed to the IoT Edge device via module twin update method.
# 1. Check the video stream to make sure the protected model is being loaded. In Azure Percept Studio, select the device and click View Stream
# 2. Check the data blob in the MM server storage account to make sure the new retraining data is being uploaded. 

In [ ]:
# The next step is to create an Azure Functions project (./azure-functions/README.md) to decrypt and send the retraining data to a Custom Vision project for model enhancement.